In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import cross_validate, TimeSeriesSplit, ShuffleSplit, StratifiedKFold, KFold, train_test_split
from sklearn.ensemble import BaggingClassifier, StackingClassifier, AdaBoostClassifier, RandomForestClassifier

In [2]:
x=np.linspace(1,100,10000) 

y=(np.sin(x)+np.random.rand(x.shape[0])*0.3>0.2).astype(int)
x = x.reshape(-1, 1)

Так как X это у нас данные, которые представляют из себя некую последовательно, то можно сделать вывод что мы работает с упорядоченными данными. Буду применять TimeSeriesSplit

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=False) #При использовании TimeSeriesSplit данные не нужно пермешивать
# По дефолту train_test_split перемешивает данные
cvsk = TimeSeriesSplit()

In [4]:
best_depth = 0
best_score = 0
for i in range(1, 100):
  clf = tree.DecisionTreeClassifier(max_depth = i)
  bagging_clf = BaggingClassifier(clf, n_estimators = 5)
  res = cross_validate(bagging_clf, x_train, y_train, cv = cvsk)
  if best_score < res['test_score'].mean():
    best_score = res['test_score'].mean()
    best_depth = i
print(f"Наилучая найденная глубина: {best_depth}")
clf = tree.DecisionTreeClassifier(max_depth = best_depth)
bagging_clf = BaggingClassifier(clf, n_estimators = 5)
bagging_clf = bagging_clf.fit(x_train, y_train)
print(f"Score моей модели = {bagging_clf.score(x_test, y_test)}")
bagging_clf_score = bagging_clf.score(x_test, y_test)

Наилучая найденная глубина: 1
Score моей модели = 0.5052


In [5]:
best_depth = 0
best_score = 0
for i in range(1, 100):
  rfc = RandomForestClassifier(max_depth=i, n_estimators=5)
  res = cross_validate(rfc, x_train, y_train, cv = cvsk)
  if best_score < res['test_score'].mean():
    best_score = res['test_score'].mean()
    best_depth = i
print(f"Наилучшая найденная глубина: {best_depth}")
rfc = RandomForestClassifier(max_depth = best_depth)
rfc.fit(x_train, y_train)
print(f"Score моей модели = {rfc.score(x_test, y_test)}")
rfc_score = rfc.score(x_test, y_test)

Наилучшая найденная глубина: 1
Score моей модели = 0.5052


In [6]:
best_depth = 0
best_score = 0
for i in range(1, 50):
  stackingClf = StackingClassifier(
    estimators=[
      ('t1', tree.DecisionTreeClassifier(max_depth=i)),
      ('t2', tree.DecisionTreeClassifier(max_depth=i)),
      ('t3', tree.DecisionTreeClassifier(max_depth=i)),
      ('t4', tree.DecisionTreeClassifier(max_depth=i)),
      ('t5', tree.DecisionTreeClassifier(max_depth=i)),
    ],
    final_estimator=tree.DecisionTreeClassifier(max_depth=i)
  )
  res = cross_validate(stackingClf, x_train, y_train, cv = cvsk)
  if best_score < res['test_score'].mean():
    best_score = res['test_score'].mean()
    best_depth = i
print(f"Наилучшая найденная глубина: {best_depth}")
stackingClf = StackingClassifier(
    estimators=[
      ('t1', tree.DecisionTreeClassifier(max_depth = best_depth)),
      ('t2', tree.DecisionTreeClassifier(max_depth = best_depth)),
      ('t3', tree.DecisionTreeClassifier(max_depth = best_depth)),
      ('t4', tree.DecisionTreeClassifier(max_depth = best_depth)),
      ('t5', tree.DecisionTreeClassifier(max_depth = best_depth)),
    ],
    final_estimator=tree.DecisionTreeClassifier(max_depth = best_depth)
  )
stackingClf.fit(x_train, y_train)
print(f"Score моей модели = {stackingClf.score(x_test, y_test)}")
stackingClf_score = stackingClf.score(x_test, y_test)

Наилучшая найденная глубина: 1
Score моей модели = 0.5052


In [7]:
best_score = 0
best_depth = 0
for i in range(1, 100):
  AdaBoost_clf = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=i), n_estimators=5)
  res = cross_validate(AdaBoost_clf, x_train, y_train, cv = cvsk)
  if best_score < res['test_score'].mean():
    best_score = res['test_score'].mean()
    best_depth = i
print(f"Наилучшая найденная глубина: {best_depth}")
AdaBoost_clf = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=best_depth), n_estimators=5)
AdaBoost_clf = AdaBoost_clf.fit(x_train, y_train)
print(f"Score моей модели = {AdaBoost_clf.score(x_test, y_test)}")
AdaBoostClf_score = AdaBoost_clf.score(x_test, y_test)


Наилучшая найденная глубина: 1
Score моей модели = 0.5052


In [8]:
x = {"AdaBoost": AdaBoostClf_score, "Bagging": bagging_clf_score, "Random forest": rfc_score, "Stacking": stackingClf_score}
scores = list(x.values())
if all(score == scores[0] for score in scores):
    print("Все модели показали одинаковый score:", scores[0])
else:
    best_model = max(x, key=x.get)
    print(f"Лучшая модель: {best_model} (score = {x[best_model]})")

Все модели показали одинаковый score: 0.5052


### Я думаю что в результате мы часто получаем скор у всех ансамблевых методов одинаковый, потому что у нас есть шум, который мешает модели хорошо обучиться. И даже увеличение глубины дерева не приводит к улучшению, так как:

- В данных всего один признак, и зависимость между $x$ и $y$ довольно простая (синусоида с шумом). Эту зависимость легко аппроксимировать даже деревом глубины 1.
- Более сложные деревья не дают прироста, потому что нет дополнительных признаков для ветвления и разделения классов.
- Если добавить больше признаков или усложнить зависимость между $x$ и $y$, то ансамбли и глубина деревьев начнут играть большую роль, и результаты моделей будут отличаться.

Таким образом, на простых данных с одним признаком ансамблевые методы не дают преимущества, и все модели выбирают минимальную глубину дерева.